In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 4.9 MB/s eta 0:00:00


In [2]:
import requests
import random
import time
import json
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import drive
import os

In [11]:

# Mount Google Drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/StockData/'

# Create folder if it doesn't exist
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

def get_stock_data():
    ticker = input("Enter the stock ticker symbol: ").strip().upper()
    url = f'https://finance.yahoo.com/quote/{ticker}'
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; win64; x64) Applewebkit/S37.36 (KHTML, Like Gecko) Chrome/127.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Applevebkit/537,36 (KHTML, Like Gecko) Chrome/125.e.e.e safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) Applewebkit/537.36 (KHTML, like Gecko) chrome/124.0.8.0 Safari/537,36'
    ]

    max_retries = 5

    for attempt in range(max_retries):
        headers = {'User-Agent': random.choice(user_agents)}

        try:
            response = requests.get(url, headers=headers)
            print(f"Status code: {response.status_code}")

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                price = soup.find('span', {'data-testid': 'qsp-price'}).text
                price_change = soup.find('span', {'data-testid': 'qsp-price-change'}).text
                price_change_percent = soup.find('span', {'data-testid': 'qsp-price-change-percent'}).text

                print("Price:", price)
                print("Change:", price_change)
                print("Change (%):", price_change_percent)

                stats_labels = [label.text for label in soup.select('div[data-testid="quote-statistics"] ul li span:first-child')]
                stats_values = [value.text for value in soup.select('div[data-testid="quote-statistics"] ul li span:last-child')]

                financial_data = dict(zip(stats_labels, stats_values))

                for label, value in financial_data.items():
                    print(f"{label}: {value}")

                data = {
                    'Stock': ticker,
                    'Price': price,
                    'Change': price_change,
                    'Change (%)': price_change_percent,
                    **financial_data
                }

                # Save data to JSON, CSV, and XLSX files in Google Drive
                with open(os.path.join(drive_path, 'stock_holder_data.json'), 'w') as json_file:
                    json.dump(data, json_file, indent=4)

                df = pd.DataFrame(list(data.items()), columns=['Data', 'Value'])
                df.to_csv(os.path.join(drive_path, 'stock_holder_data.csv'), index=False)
                df.to_excel(os.path.join(drive_path, 'stock_holder_data.xlsx'), index=False)

                print("Download links:")
                print(f"JSON: {drive_path}stock_holder_data.json")
                print(f"CSV: {drive_path}stock_holder_data.csv")
                print(f"XLS: {drive_path}stock_holder_data.xlsx")

                return data

            elif response.status_code == 429:
                print("Rate limited. Waiting before retrying...")
                time.sleep(10 * (attempt + 1))

            else:
                print(f"Unexpected status code: {response.status_code}")
                break

        except Exception as e:
            print(f"Error occurred: {e}")

    print("Failed to fetch data after retries.")
    return {}

data = get_stock_data()


Mounted at /content/drive
Enter the stock ticker symbol: TSLA
Status code: 200
Price: 272.36 
Change: -12.29 
Change (%): (-4.32%) 
Previous Close: 284.65 
Open: 270.80 
Bid: 271.93 x 200
Ask: 272.90 x 300
Day's Range: 261.84 - 278.28 
52 Week Range: 138.80 - 488.54 
Volume: 73,611,607 
Avg. Volume: 84,898,286 
Market Cap (intraday): 901.68B 
Beta (5Y Monthly): 2.51
PE Ratio (TTM): 132.86 
EPS (TTM): 2.05 
Earnings Date: Apr 23, 2025 - May 3, 2025
Forward Dividend & Yield: --
Ex-Dividend Date: --
1y Target Est: 342.94 
Download links:
JSON: /content/drive/My Drive/StockData/stock_holder_data.json
CSV: /content/drive/My Drive/StockData/stock_holder_data.csv
XLS: /content/drive/My Drive/StockData/stock_holder_data.xlsx
